In [3]:
import numpy as np
from PIL import ImageGrab
import cv2
import time

In [4]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
game_url = "game/dino.html"
chrome_driver_path = "../chromedriver.exe"
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.set_window_size(200, 300)
        self._driver.get(os.path.abspath(game_url))
        if custom_config:
            self._driver.execute_script("Runner.config.ACCELERATION=0")
    def get_crashed(self):
        return self._driver.execute_script("return Runner.instance_.crashed")
    def get_playing(self):
        return self._driver.execute_script("return Runner.instance_.playing")
    def restart(self):
        return self._driver.execute_script("Runner.instance_.restart()")
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def press_down(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_DOWN)
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array)
        return int(score)
    def end(self):
        self._driver.close()

In [5]:
class DinoAgent:
    def __init__(self,game):
        self._game = game;
        self.jump();
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def jump(self):
        self._game.press_up()
    def duck(self):
        self._game.press_down()

In [6]:
#processing image as required
def process_img(image):
    #image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)qqq
    #game is already in grey scale canvas, canny to get only edges and reduce unwanted objects(clouds)
    image = cv2.Canny(image, threshold1 = 100, threshold2 = 200)
    #image = resized_image = cv2.resize(image, (80, 80)) 
    image = cv2.resize(image, (0,0), fx = 0.50, fy = 0.50)
    #image = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,21)
    return  image

In [7]:
def grab_screen():
    screen =  np.array(ImageGrab.grab(bbox=(0,180,400,400)))
    image = process_img(screen)
    return image

In [8]:
print(grab_screen().shape)
# game = Game()
# dino = DinoAgent(game)
# last_time = time.time()
# while(True):
    
# #     print('loop took {} seconds'.format(time.time()-last_time))
# #     last_time = time.time()
# #     cv2.imwrite("./img_data/dino"+str(time())+".jpg",image)
# #     dino.duck()
#     #exit on q pres
#     image,r_t,end_t = get_state(game,dino,2)
# #     print('{0} {1} '.format(r_t,end_t))
#     #cv2.imshow('window',image)
#     if(dino.is_crashed()):
#         #jumping starts the game again if dino has crashed
#         print(game.get_score())
#         game.restart()
        
#     if (cv2.waitKey(25) & 0xFF == ord('q')):
#         cv2.destroyAllWindows()
#         game.end()
#         cv2.imwrite('dino.jpg',image)
#         break

(110, 200)


In [9]:
#game parameters
ACTIONS = 3 # possible actions: jump, duck , do nothing
GAMMA = 0.99 # decay rate of past observations
OBSERVATION = 3200. # timesteps to observe before training
EXPLORE = 3000000. # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4

In [10]:
img_rows , img_cols = 200, 110
img_channels = 4 #We stack 4 frames

In [11]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(3))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    print("We finish building the model")
    return model

In [ ]:
def trainNetwork(model,game_state):
    # open up a game state to communicate with emulator

    # store the previous observations in replay memory
    D = deque()
    display = show_img()
    display.__next__()
    # get the first state by doing nothing and preprocess the image to 80x80x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 
    x_t, r_0, terminal = game_state.get_state(do_nothing)
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    

    #In Keras, need to reshape
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*80*80*4

    print ("this is the shape"+str(s_t.shape))

    if True :#args['mode'] == 'Run':
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = INITIAL_EPSILON

    t = 0
    while (True):
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0
        a_t = np.zeros([ACTIONS])
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0:
            if random.random() <= epsilon:
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else:
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)
                action_index = max_Q
                a_t[action_index] = 1

        #We reduced the epsilon gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        display.send(x_t1)
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x80x80x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)

        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE:
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)



            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 80, 80, 4
            print (inputs.shape)
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]
                state_t1 = minibatch[i][3]
                terminal = minibatch[i][4]
                # if terminated, only equals reward

                inputs[i:i + 1] = state_t    #I saved down s_t

                targets[i] = model.predict(state_t)  # Hitting each buttom probability
                Q_sa = model.predict(state_t1)

                if terminal:
                    targets[i, action_t] = reward_t
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            # targets2 = normalize(targets)
            loss += model.train_on_batch(inputs, targets)

        s_t = s_t1
        t = t + 1

        # save progress every 10000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            model.save_weights("model.h5", overwrite=True)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)

        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state, \
            "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t, \
            "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [13]:
def playGame():
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)
    model = buildmodel()
    trainNetwork(model,game_state)

In [24]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
    def get_state(self,actions):
        
        reward = 1
        is_over = False
        if actions[1] == 1:
            self._agent.jump()
            reward = 0.1
        elif (actions[2] == 1):
#             self._agent.duck()
            self._agent.jump()
            reward = 0.1
        if self._agent.is_crashed():
            self._game.restart()
            reward = -1
            is_over = True
        image = grab_screen()
        return image, reward, is_over

In [15]:

from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD , Adam
import tensorflow as tf
from collections import deque
import random
import json

Using TensorFlow backend.


In [25]:
playGame()

Now we build the model
We finish building the model
this is the shape(1, 110, 200, 4)
----------Random Action----------


C:\Users\ravi7\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(110, 200,..., strides=(4, 4), padding="same")`
  after removing the cwd from sys.path.
C:\Users\ravi7\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), strides=(2, 2), padding="same")`
  
C:\Users\ravi7\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same")`
  


TIMESTEP 1 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 2 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 4 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 5 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 6 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 1 / Q_MAX  0 / Loss  0
TIMESTEP 7 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 8 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 9 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 1 / Q_MAX  0 / Loss  0
TIMESTEP 10 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 11 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 12 / STATE observe / EPSILON 0.1 / ACTION 2 / R

TIMESTEP 93 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 94 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 95 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 96 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 97 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMESTEP 98 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 99 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 1 / Q_MAX  0 / Loss  0
TIMESTEP 100 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 101 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD -1 / Q_MAX  0 / Loss  0
TIMESTEP 102 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 103 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
--------

TIMESTEP 184 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMESTEP 185 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 186 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMESTEP 187 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 188 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 1 / Q_MAX  0 / Loss  0
TIMESTEP 189 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 190 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 191 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -1 / Q_MAX  0 / Loss  0
TIMESTEP 192 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 193 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 194 / STATE observe / EPSILON 0.1 / ACTION 2 /

...

TIMESTEP 1180 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1181 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMESTEP 1182 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1183 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1184 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1185 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1186 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1187 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1188 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1189 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1190 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX 

TIMESTEP 1269 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1270 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1271 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1272 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1273 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 1 / Q_MAX  0 / Loss  0
TIMESTEP 1274 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1275 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1276 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1277 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1278 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1279 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1280 / STATE o

TIMESTEP 1358 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1359 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMESTEP 1360 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1361 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1362 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1363 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1364 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1365 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1366 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1367 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD -1 / Q_MAX  0 / Loss  0
TIMESTEP 1368 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  

TIMESTEP 1449 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1450 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1451 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 1 / Q_MAX  0 / Loss  0
TIMESTEP 1452 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1453 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 1 / Q_MAX  0 / Loss  0
TIMESTEP 1454 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD -1 / Q_MAX  0 / Loss  0
TIMESTEP 1455 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1456 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1457 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1458 / STATE observe / EPSILON 0.1 / ACTION 2 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1459 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 1 / Q_MAX  0 / Loss  0
TIMESTEP 1460 / STATE observ

StopIteration: 

In [19]:
from matplotlib import pyplot as plt
def show_img():
    """
    Coroutine to store images in the "images" directory
    """
    frame = 0
    while True:
        screen = (yield)
        cv2.imshow("preview", screen)
        if (cv2.waitKey(25) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

        frame += 1